In [2]:
# # preliminary installs and mounting of google drive
#######################################################
#!pip install transformers
#!pip install ipynb
#!pip install google.colab
#from google.colab import drive
#drive.mount('/content/gdrive', force_remount=True)
#!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_sm-0.2.4.tar.gz

In [3]:
#!pip install google.colab

In [4]:
# !pip install transformers
# !pip install ipynb
# !pip install nltk
# !pip install gensim
# !pip install scispacy
# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_md-0.2.4.tar.gz

In [5]:
## initializing path (are we on colab or azure machine?)
import os 
ipynb_path = '/Users/lilifang/KCL/KCL_Angus/CRIS_COVID_Aurelie'
#T:\aurelie_mascio\BERT_models\violence

In [6]:
# !pip install torch
# !pip install transformers -U
# !pip install -U pip
# !pip install packaging==20.9

In [7]:
#!pip install pytorch-transformers
# !pip install --upgrade torch
# !pip install --upgrade torchvision
# !pip install --upgrade torchtext
# !pip install --upgrade torchaudio

In [8]:
#!pip install torch==1.4.0

In [9]:
import io
import sys
import pandas as pd
import numpy as np
from tqdm import tqdm, trange
from collections import Counter
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence
from transformers import BertTokenizer, BertConfig, BertForSequenceClassification, AdamW
import matplotlib.pyplot as plt

/opt/anaconda3/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.24.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
2022-11-24 14:01:15.154286: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
#!pip install transformers

In [14]:
print(torch.__version__)
print(np.__version__)
print(pd.__version__)
print(tqdm.__version__)
print(transformers.__version__)

1.13.0
1.23.5
1.4.4


AttributeError: type object 'tqdm' has no attribute '__version__'

In [22]:
#from transformers import AutoTokenizer

In [23]:
#from transformers import BertTokenizer

In [24]:

# sys.path.insert(0, ipynb_path)
# from ipynb.fs.full.NLP_utils import *


In [25]:
#%run NLP_utils.ipynb

In [26]:
def convert_to_cat(labels, binary=False):
    if not isinstance(labels, pd.Series):
        labels = pd.Series(labels)
    if binary:
        main_val = labels.mode()[0]
        other_val = [x for x in labels.unique() if x!= main_val]
        labels = np.where(labels!=main_val,1,0)
        corresp = pd.DataFrame([main_val, 'other'], columns=['old_label'])
    elif np.issubdtype(labels.dtype, np.number) and (labels.min() >= 0):
        print('labels already in a good format')
        return {'labels':labels, 'categories':pd.DataFrame(labels.unique())}
    else:
        cats = labels.astype('category').cat
        labels = cats.codes.astype('long') # convert annotations to integers
        corresp = pd.DataFrame(cats.categories, columns=['old_label'])
    corresp.index.rename('new_label', inplace=True)
    print('labels have been transformed for the model:\n\n', corresp)
    return {'labels':labels, 'categories':corresp}

In [27]:
# TO RUN K-FOLD VALIDATION
def run_KFOLD(dataset, base_model, model_trainer, base_model_loader=None,
             n_splits=10, random_state=42, n_epochs=5, **kwargs):
    from torch.utils.data import DataLoader 
    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    labels = pd.Series([1] * len(dataset))
    # tracking variables
    best_f1, fold_nb = 0, 0
    stats, stats_classes = pd.DataFrame(), pd.DataFrame()
    # run k fold
    for train_ix, test_ix in kfold.split(labels, labels):
        fold_nb +=1
        # need to load each time otherwise remembers training from previous fold
        if base_model_loader is not None:
            base_model_tmp = base_model_loader(base_model, **kwargs)
        else:
            base_model_tmp = base_model
        print('####################### RUNNING FOLD:', fold_nb)
        train_dataset = torch.utils.data.Subset(dataset, train_ix)
        val_dataset = torch.utils.data.Subset(dataset, test_ix)
        print(type(train_dataset), ' train set:', len(train_dataset), ' test set:',len(val_dataset))
        train_dataloader, validation_dataloader = create_dataloader(train_dataset, val_dataset)
        del train_dataset, val_dataset
        print('training and evaluating model')
        res = model_trainer(base_model_tmp, train_dataloader, validation_dataloader, n_epochs=n_epochs, output_dir=None)
        del base_model_tmp, train_dataloader, validation_dataloader
        # store perf metrics and model
        stats_tmp = pd.DataFrame.from_dict(res['stats'],orient='index', columns=['value'])
        stats_tmp['fold'] = fold_nb
        stats = pd.concat([stats, stats_tmp])
        res['stats_classes']['fold'] = fold_nb
        stats_classes = pd.concat([stats_classes, res['stats_classes']])
        if res['stats']['f1'] >= best_f1:
            best_f1 = res['stats']['f1']
            res_to_save = res
        del res
            
        
    print('best F1 score obtained across splits: {:.3f}'.format(best_f1))
    return {'stats':stats, 'stats_classes':stats_classes, 'model':res_to_save['model']}

In [28]:
# Create the DataLoaders for our training and validation sets.
# For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32.
def create_dataloader(train_dataset, val_dataset, batch_size = 32):
    # We'll take training samples in random order. 
    train_dataloader = DataLoader(
                train_dataset,  # The training samples.
                sampler = RandomSampler(train_dataset), # Select batches randomly
                batch_size = batch_size # Trains with this batch size.
            )
    # For validation the order doesn't matter, so we'll just read them sequentially.
    validation_dataloader = DataLoader(
                val_dataset, # The validation samples.
                sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
                batch_size = batch_size # Evaluate with this batch size.
            )
    
    return [train_dataloader, validation_dataloader]

# FUNCTIONS TO PREP DATASET / TRAIN BERT

In [29]:
def prep_BERT_dataset(sentences, labels=None, BERT_tokenizer='bert-base-uncased', MAX_TKN_LEN=511, debug=False):
    sentences = pd.Series(sentences)
    # load relevant data and add special tokens for BERT to work properly
    sentences = ["[CLS] " + query + " [SEP]" for query in sentences]
    if labels is not None:
        labels = convert_to_cat(labels, binary=False)['labels']
    else:
        labels = pd.Series([1] * len(sentences))
    if debug: print(sentences[0])
    
    # Tokenize with BERT tokenizer
    tokenizer = BertTokenizer.from_pretrained(BERT_tokenizer, do_lower_case=True)
    # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    # seems there are issues for sequence lengths > 512. TO CHECK
    # https://github.com/huggingface/transformers/issues/2446
    # !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
    if MAX_TKN_LEN is not None:
        print('cutting the length of tokens to', MAX_TKN_LEN)
        tokenized_texts = [tokenizer.tokenize(sent)[0:511] for sent in sentences]
    else:
        tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
    input_ids = [tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts]
    if debug:
        print ("Tokenize the first sentence:")
        print (len(tokenized_texts), len(input_ids), len(input_ids[0]))
        print (tokenized_texts[0], input_ids[0])
    
    # add paddding to input_ids
    input_ids_padded = pad_sequence([torch.tensor(i) for i in input_ids]).transpose(0,1)
    if debug: print(input_ids_padded.size(), len(input_ids_padded))
    
    # Create attention masks
    attention_masks = []
    # Create a mask of 1s for each token followed by 0s for padding
    for seq in input_ids_padded:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)
    
    # create dataset
    dataset = TensorDataset(input_ids_padded, torch.tensor(attention_masks), torch.tensor(labels))
    num_labels = labels.nunique()
    return {'dataset': dataset, 'num_labels': num_labels}

In [30]:
def run_BERT(model, train_dataloader, validation_dataloader, n_epochs=5, output_dir=None):
    ###################################################################################
    # BERT fine-tuning parameters
    device = get_device()
    try:
        model.cuda()
    except:
        device = None
        print('using CPU, this will be slow!')
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
                                    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
                                    'weight_decay_rate': 0.01},
                                    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                                    'weight_decay_rate': 0.0}
                                    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=2e-5, eps = 1e-8)

    # BERT training loop
    train_loss_set = []
    best_f1, best_epoch = 0, 0
    for _ in trange(n_epochs, desc="Epoch"):
        ###################################################################################
        ## TRAINING

        # Set our model to training mode
        model.train()  
        # Tracking variables
        tr_loss, tr_perf, tr_perf_classes = 0, Counter({}), pd.DataFrame()
        running_len = 0
        # Train the data for one epoch
        for step, batch in enumerate(train_dataloader):
            # Unpack the inputs from our dataloader (and move to GPU if using)
            batch = batch_to_gpu(batch, device)
            b_input_ids, b_input_mask, b_labels = batch
            # Clear out the gradients (by default they accumulate)
            optimizer.zero_grad()
            # Forward pass
            output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            loss = output[0]
            logits = output[1]
            print(loss)
            train_loss_set.append(loss.item())    
            # Backward pass
            loss.backward()
            # Update parameters and take a step using the computed gradient
            optimizer.step()
            # Update tracking variables
            tr_loss += loss.item()
            print('test')
            tmp_tr_perf = perf_metrics(to_cpu(logits), to_cpu(b_labels), average='weighted')
            tmp_tr_perf.update((k,v*len(b_input_ids)) for k,v in tmp_tr_perf.items())
            running_len +=len(b_input_ids)
            tr_perf = tr_perf + Counter(tmp_tr_perf)
            tmp_tr_perf_classes = perf_metrics_classes(to_cpu(logits), to_cpu(b_labels))
            tr_perf_classes = pd.concat((tr_perf_classes, tmp_tr_perf_classes))
        
        #print('classes detail \n\n', tr_perf_classes)
        tr_perf = {k:v/running_len for k,v in tr_perf.items()}
        tr_perf_classes[['f1-score', 'precision', 'recall']] = tr_perf_classes[['f1-score', 'precision', 'recall']].multiply(tr_perf_classes['support'], axis="index")
        tr_perf_classes=tr_perf_classes.groupby(tr_perf_classes.index).sum()
        tr_perf_classes[['f1-score','precision', 'recall']] = tr_perf_classes[['f1-score', 'precision', 'recall']].div(tr_perf_classes['support'], axis="index")
        #tr_perf_classes=tr_perf_classes.replace(0, np.NaN).groupby(tr_perf_classes.index).agg({'f1-score':'mean','precision':'mean', 'recall':'mean', 'support':'sum'})
        print('TRAIN - Loss: {:.3f} - F1: {:.3f} Acc: {:.3f} P: {:.3f} R: {:.3f}'.format(tr_loss/(1+step), tr_perf['f1'], tr_perf['acc'], tr_perf['p'], tr_perf['r']))

        ###################################################################################
        ## VALIDATION

        # Put model in evaluation mode
        model.eval()
        # Tracking variables 
        eval_perf, eval_perf_classes = Counter({}), pd.DataFrame()
        running_len = 0
        # Evaluate data for one epoch
        for step, batch in enumerate(validation_dataloader):
            # Unpack the inputs from our dataloader (and move to GPU if using)
            batch = batch_to_gpu(batch, device)
            b_input_ids, b_input_mask, b_labels = batch       
            with torch.no_grad(): # Telling the model not to compute or store gradients, saving memory and speeding up validation
                # Forward pass, calculate logit predictions
                output= model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)  
                loss = output[0]
                logits = output[1]
            # Update tracking variables 
            tmp_eval_perf = perf_metrics(to_cpu(logits), to_cpu(b_labels), average='weighted')
            tmp_eval_perf.update((k,v*len(b_input_ids)) for k,v in tmp_eval_perf.items())
            #print('STEP:', step, 'LEN', len(b_input_ids), tmp_eval_perf)
            running_len +=len(b_input_ids)
            eval_perf = eval_perf + Counter(tmp_eval_perf)
            tmp_eval_perf_classes = perf_metrics_classes(to_cpu(logits), to_cpu(b_labels))
            eval_perf_classes = pd.concat((eval_perf_classes, tmp_eval_perf_classes))
        
        eval_perf = {k:v/running_len for k,v in eval_perf.items()}  #eval_perf = {k:v/(1+step) for k,v in eval_perf.items()}
        eval_perf_classes[['f1-score', 'precision', 'recall']] = eval_perf_classes[['f1-score', 'precision', 'recall']].multiply(eval_perf_classes['support'], axis="index")
        eval_perf_classes=eval_perf_classes.groupby(eval_perf_classes.index).sum()
        eval_perf_classes[['f1-score','precision', 'recall']] = eval_perf_classes[['f1-score', 'precision', 'recall']].div(eval_perf_classes['support'], axis="index")
        #eval_perf_classes=eval_perf_classes.replace(0, np.NaN).groupby(eval_perf_classes.index).agg({'f1-score':'mean','precision':'mean', 'recall':'mean', 'support':'sum'})
        print('TEST -- F1: {:.3f} Acc: {:.3f} P: {:.3f} R: {:.3f}'.format(eval_perf['f1'], eval_perf['acc'], eval_perf['p'], eval_perf['r']))
        
        # store perf metrics and model
        if eval_perf['f1']>=best_f1:
            best_f1 = eval_perf['f1']
            best_epoch = _ + 1
            stats_to_save = eval_perf
            tr_perf_classes['dataset'] = 'train'
            eval_perf_classes['dataset'] = 'test'
            stats_classes_to_save = pd.concat([tr_perf_classes, eval_perf_classes])
            model_to_save = model
        print('best F1 score obtained: {:.3f} at epoch {}'.format(best_f1, best_epoch))

    # save model with best f1
    if output_dir is not None:
        try:
            print('saving model...')
            model_to_save.save_pretrained(output_dir)
            stats_classes_to_save.to_csv(output_dir+'/stats.csv', header=True)
        except:
            print('model not saved, please enter valid path')
        
    return {'stats':stats_to_save, 'stats_classes':stats_classes_to_save, 'model':model_to_save}


# Functions to train/evaluate model

In [31]:
# output_dir='/home/ubuntu/data/ACL2020/bert_models/base')
def train_BERT(sentences, labels, BERT_tokenizer='bert-base-uncased', test_size=0.1, 
               n_epochs=5, batch_size=32, output_dir=None, MAX_TKN_LEN=511):
    print('formating dataset')
    prep_data = prep_BERT_dataset(sentences=sentences, labels=labels, BERT_tokenizer=BERT_tokenizer, MAX_TKN_LEN=MAX_TKN_LEN)
    dataset = prep_data['dataset']
    num_labels = prep_data['num_labels']
    # split into train/test
    print('splitting in train/test sets')
    test_len= int(len(dataset)*test_size)
    train_len = len(dataset) - test_len
    print('test set:', test_len, 'train set:', train_len)
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_len, test_len])
    # Create the DataLoaders for our training and validation sets.
    train_dataloader, validation_dataloader = create_dataloader(train_dataset, val_dataset, batch_size = batch_size)
    # Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 
    print('loading pre-trained BERT')
    pretrained_model = BertForSequenceClassification.from_pretrained(BERT_tokenizer, num_labels=num_labels)
    # train and evaluate BERT
    print('training BERT')
    res = run_BERT(pretrained_model, train_dataloader, validation_dataloader, n_epochs=n_epochs, output_dir=output_dir)
    return res

In [32]:
def BERT_KFOLD(sentences, labels, BERT_tokenizer='bert-base-uncased',
               n_splits=10, random_state=42, n_epochs=5, output_dir=None, MAX_TKN_LEN=511):
    prep_data = prep_BERT_dataset(sentences=sentences, labels=labels, BERT_tokenizer=BERT_tokenizer, MAX_TKN_LEN=MAX_TKN_LEN)
    dataset = prep_data['dataset']
    num_labels = prep_data['num_labels']
    pretrained_model = BertForSequenceClassification.from_pretrained(BERT_tokenizer, num_labels=num_labels)
    res = run_KFOLD(dataset=dataset, base_model=BERT_tokenizer, model_trainer=run_BERT, base_model_loader= BertForSequenceClassification.from_pretrained,
                    n_splits=n_splits, random_state=random_state, n_epochs=n_epochs, num_labels=prep_data['num_labels'])
    if output_dir is not None:
        try:
            print('saving model in:', output_dir)
            res['model'].save_pretrained(output_dir)
            res['stats_classes'].to_csv(output_dir+'/stats.csv', header=True)
        except:
            print('model not saved, please enter valid path')    
    return res

# Function to load saved model and classify new data

In [33]:
def load_BERT_components(trained_bert_model, BERT_tokenizer='bert-base-uncased', do_lower_case=True):
    tokenizer = BertTokenizer.from_pretrained(BERT_tokenizer, do_lower_case=do_lower_case)
    trained_bert_model = BertForSequenceClassification.from_pretrained(trained_bert_model)
    return {'tokenizer':tokenizer, 'model':trained_bert_model}

In [34]:
# load pre-trained model and classify a new sentence
def load_and_run_BERT(sentences, trained_bert_model, BERT_tokenizer='bert-base-uncased', MAX_TKN_LEN=511, batch_size=32):
    if isinstance(BERT_tokenizer, str): # load BERT base model if needed
        trained_bert_model = BertForSequenceClassification.from_pretrained(trained_bert_model)
    preds_class, probs, preds = [], [], pd.DataFrame()
    sentences = pd.Series(sentences)
    sentences_dataset = prep_BERT_dataset(sentences, labels=None, BERT_tokenizer=BERT_tokenizer, MAX_TKN_LEN=MAX_TKN_LEN)['dataset']
    # b_input_ids, b_input_mask, b_labels = sentences_dataset.tensors
    validation_dataloader = DataLoader(sentences_dataset, sampler=SequentialSampler(sentences_dataset), batch_size=batch_size)
    for step, batch in enumerate(validation_dataloader):
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            output= trained_bert_model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels, return_dict = False)
            loss = output[0]
            logits = output[1]
        preds_curr = logits.detach().numpy()
        preds = preds.append(pd.DataFrame(preds_curr), ignore_index=True)
        probs = np.append(probs, np.max(np.exp(preds_curr)/(1+np.exp(preds_curr)), axis=1))
        preds_class = np.append(preds_class, np.argmax(preds_curr, axis=1).flatten())
    
    # put results in nice format
    preds['sentences'] = sentences
    preds['preds'] = preds_class
    preds['probs'] = probs
    return preds

## TESTING

def test():
    ####################################################################################
    # 0. load pretrained BERT
    ####################################################################################
    #BERT_tokenizer = 'bert-base-uncased'
    #BERT_tokenizer = 'emilyalsentzer/Bio_ClinicalBERT'
    BERT_tokenizer = 'monologg/biobert_v1.0_pubmed_pmc'

    ####################################################################################
    # 1. Load Dataset
    ####################################################################################
    df = pd.read_csv('mimic_status_10folds.csv')[0:100]
    #df = pd.read_csv('/home/zeljko_kraljevic/data/F20/annotations/attention_10fold.csv')
    #df = pd.read_excel('/home/ZKraljevic/data/covid_anxiety/anxiety_batch_1_riley.xlsx')[0:100]
    #df= pd.read_excel('anxiety_batch_2_riley.xlsx')[0:100]
    text_col = 'clean_text'
    label_col = 'annotation'
    df[label_col] = df[label_col].fillna('irrelevant').replace({'irrelevant':0, 'affirmed':1, 'negated':0})
    print(df.head())

    ####################################################################################
    # 2. To run K-fold
    ####################################################################################
    test = BERT_KFOLD(sentences=df[text_col], labels=df[label_col], n_splits=10, BERT_tokenizer=BERT_tokenizer, n_epochs=5, random_state=666)
    #test['model'].save_pretrained('/home/ZKraljevic/data/covid_anxiety/bert_models')
    test['model'].save_pretrained('/home/aurelie_mascio/bert_models')

    ####################################################################################
    # 2b. To run 1 simulation
    ####################################################################################
    test = train_BERT(sentences=df[text_col], labels=df[label_col], BERT_tokenizer='bert-base-uncased', test_size=0.1, n_epochs=1)
    
    ####################################################################################
    # 3. To test on new data
    ####################################################################################
    sentences = df.head(5)[text_col] # put your new sentences here
    print(sentences)
    bert_model_path = 'gdrive/My Drive/Colab Notebooks/prometheus/datasets/bert_models' # '/home/ubuntu/ZKraljevic/covid_anxiety/bert_models/base'
    load_and_run_BERT(sentences, trained_bert_model=bert_model_path, BERT_tokenizer='bert-base-uncased')

In [35]:
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader   

In [36]:
def get_device():
    # specify GPU device
    try:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        if device != 'cpu':
            print('number GPUs used:',torch.cuda.device_count())
            print('device name:', torch.cuda.get_device_name(0))
    except:
        device = None
        print('no CUDA capable device detected')
    return device

In [37]:
# !pip install openpyxl

In [38]:
def batch_to_gpu(batch, device=None):
    if device is None:
        return batch
    try: # we're using a GPU
        batch = tuple(t.to(device) for t in batch)
    except:
        batch = tuple(t for t in batch)
    return batch

In [40]:
# def perf_metrics(preds, labels, average='weighted', debug=False):
#     try:
#         pred_flat = np.argmax(preds, axis=1).flatten()
#         #pred_flat = torch.max(pred_vec, 1)[1]
#     except:
#         print('only 1 dimension in labels prediction, no need for argmax')
#         pred_flat = preds.flatten()
#     labels_flat = labels.flatten()
#     acc = accuracy_score(labels_flat, pred_flat)
#     f1 = f1_score(labels_flat, pred_flat, average=average)
#     p = precision_score(labels_flat, pred_flat, average=average)
#     r = recall_score(labels_flat, pred_flat, average=average)
#     if debug:
#         print("PERF -- Acc: {:.3f} F1: {:.3f} Precision: {:.3f} Recall: {:.3f}".format(acc,f1,p,r))
#     return {'f1':f1, 'acc':acc, 'p':p, 'r':r}


In [41]:
# Function to calculate performance of our predictions vs labels
def perf_metrics(preds, labels, average='weighted', debug=False):
    try:
        pred_flat = np.argmax(preds, axis=1).flatten()
        #pred_flat = torch.max(pred_vec, 1)[1]
    except:
        print('only 1 dimension in labels prediction, no need for argmax')
        pred_flat = preds.flatten()
    labels_flat = labels.flatten()
    acc = accuracy_score(labels_flat, pred_flat)
    f1 = f1_score(labels_flat, pred_flat, average=average)
    p = precision_score(labels_flat, pred_flat, average=average)
    r = recall_score(labels_flat, pred_flat, average=average)
    if debug:
        print("PERF -- Acc: {:.3f} F1: {:.3f} Precision: {:.3f} Recall: {:.3f}".format(acc,f1,p,r))
    return {'f1':f1, 'acc':acc, 'p':p, 'r':r}

def to_cpu(vec, detach=True):
    try:
        vec = vec.detach().cpu().numpy() if detach else vec.to('cpu').numpy()
    except:
        vec = vec.detach().numpy() if detach else vec.numpy()
    return vec
def perf_metrics_classes(preds, labels):
    try:
        pred_flat = np.argmax(preds, axis=1).flatten()
    except:
        print('only 1 dimension in labels prediction, no need for argmax')
        pred_flat = preds.flatten()
    labels_flat = labels.flatten()
    report = classification_report(labels_flat, pred_flat, output_dict=True)
    df = pd.DataFrame(report).sort_index().transpose()
    return df

In [45]:
####################################################################################
# 0. load pretrained BERT
####################################################################################
#BERT_tokenizer = 'bert-base-uncased'
#BERT_tokenizer = 'emilyalsentzer/Bio_ClinicalBERT'
#BERT_tokenizer = 'monologg/biobert_v1.0_pubmed_pmc'
####################################################################################

# 1. Load Dataset
####################################################################################
#df = pd.read_csv('/home/zeljko_kraljevic/data/F20/annotations/attention_10fold.csv')
#df = pd.read_excel('/home/ZKraljevic/data/covid_anxiety/anxiety_batch_1_riley.xlsx')[0:100]
df= pd.read_excel('MIMIC_sexual_violence.xlsx')
print(df.columns)


Index(['annotation', 'clean_text'], dtype='object')


In [47]:
text_col = 'clean_text'
label_col = 'annotation'
df[label_col] = df[label_col].fillna('irrelevant').replace({'no':0, 'yes':1, 'negated':0})
print(df.head())
print(df.shape)
print(df.groupby('annotation').count())

  annotation                                         clean_text
0      Other  The patient gone through a lot of emotional an...
1      Other  Psychology to help someone deal with the traum...
2   Affirmed  the patient sited emotional, physical abuse, a...
3   Affirmed  he said about his abusive experiences, and als...
4   Affirmed  The patient mention that the during his childh...
(29, 2)
            clean_text
annotation            
Affirmed            17
Other               12


In [48]:
from transformers import AutoTokenizer, AutoModelWithLMHead, AutoModel
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
bert = AutoModel.from_pretrained("monologg/biobert_v1.0_pubmed_pmc")

Some weights of the model checkpoint at monologg/biobert_v1.0_pubmed_pmc were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [49]:
####################################################################################
# 2. To run K-fold
####################################################################################
BERT_tokenizer = 'monologg/biobert_v1.0_pubmed_pmc'
sentences=df[text_col]
labels=df[label_col]
n_splits=5
BERT_tokenizer=BERT_tokenizer
n_epochs=3
random_state=666
output_dir=None
MAX_TKN_LEN=511
debug=False

# load relevant data and add special tokens for BERT to work properly
sentences = ["[CLS] " + query + " [SEP]" for query in sentences]
if labels is not None:
    labels = convert_to_cat(labels, binary=False)['labels']
else:
    labels = pd.Series([1] * len(sentences))
if debug: print(sentences[0])

# Tokenize with BERT tokenizer
tokenizer = BertTokenizer.from_pretrained(BERT_tokenizer, do_lower_case=True)
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# seems there are issues for sequence lengths > 512. TO CHECK
# https://github.com/huggingface/transformers/issues/2446
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
if MAX_TKN_LEN is not None:
    print('cutting the length of tokens to', MAX_TKN_LEN)
    tokenized_texts = [tokenizer.tokenize(sent)[0:511] for sent in sentences]
else:
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
input_ids = [tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts]
if debug:
    print ("Tokenize the first sentence:")
    print (len(tokenized_texts), len(input_ids), len(input_ids[0]))
    print (tokenized_texts[0], input_ids[0])

# add paddding to input_ids
input_ids_padded = pad_sequence([torch.tensor(i) for i in input_ids]).transpose(0,1)
if debug: print(input_ids_padded.size(), len(input_ids_padded))

# Create attention masks
attention_masks = []
# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids_padded:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

# create dataset
dataset = TensorDataset(input_ids_padded, torch.tensor(attention_masks), torch.tensor(labels))
num_labels = labels.nunique()


labels have been transformed for the model:

           old_label
new_label          
0          Affirmed
1             Other
cutting the length of tokens to 511


In [50]:
prep_data={'dataset': dataset, 'num_labels': num_labels}
dataset = prep_data['dataset']
num_labels = prep_data['num_labels']
pretrained_model = BertForSequenceClassification.from_pretrained(BERT_tokenizer, num_labels=num_labels)


Some weights of the model checkpoint at monologg/biobert_v1.0_pubmed_pmc were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the mode

In [51]:
res = run_KFOLD(dataset=dataset, base_model=BERT_tokenizer, model_trainer=run_BERT, base_model_loader= BertForSequenceClassification.from_pretrained, n_splits=n_splits, random_state=random_state, n_epochs=n_epochs, num_labels=prep_data['num_labels'])


Some weights of the model checkpoint at monologg/biobert_v1.0_pubmed_pmc were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the mode

####################### RUNNING FOLD: 1
<class 'torch.utils.data.dataset.Subset'>  train set: 23  test set: 6
training and evaluating model
number GPUs used: 0
no CUDA capable device detected
using CPU, this will be slow!


Epoch:   0%|                                              | 0/3 [00:00<?, ?it/s]

tensor(0.7272, grad_fn=<NllLossBackward0>)
test
TRAIN - Loss: 0.727 - F1: 0.340 Acc: 0.348 P: 0.333 R: 0.348


Epoch:  33%|████████████▋                         | 1/3 [00:05<00:11,  5.80s/it]

...

TEST -- F1: 0.450 Acc: 0.600 P: 0.360 R: 0.600
best F1 score obtained: 0.450 at epoch 1
tensor(0.6777, grad_fn=<NllLossBackward0>)
test
TRAIN - Loss: 0.678 - F1: 0.583 Acc: 0.583 P: 0.583 R: 0.583


/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Epoch:  67%|█████████████████████████▎            | 2/3 [00:11<00:05,  5.78s/it]

TEST -- F1: 0.450 Acc: 0.600 P: 0.360 R: 0.600
best F1 score obtained: 0.450 at epoch 2
tensor(0.6374, grad_fn=<NllLossBackward0>)
test
TRAIN - Loss: 0.637 - F1: 0.682 Acc: 0.708 P: 0.732 R: 0.708


/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Epoch: 100%|██████████████████████████████████████| 3/3 [00:17<00:00,  5.82s/it]


TEST -- F1: 0.450 Acc: 0.600 P: 0.360 R: 0.600
best F1 score obtained: 0.450 at epoch 3
best F1 score obtained across splits: 0.815


In [52]:
output_dir= '/Users/lilifang/KCL/KCL_Angus/CRIS_COVID_Aurelie/MiMiC_sexual_model_LF'

try:
    print('saving model in:', output_dir)
    res['model'].save_pretrained(output_dir)
    res['stats_classes'].to_csv(output_dir+'/stats.csv', header=True)
except:
    print('model not saved, please enter valid path')


saving model in: /Users/lilifang/KCL/KCL_Angus/CRIS_COVID_Aurelie/MiMiC_sexual_model_LF
